## Some preliminary checks

In [1]:
import torch
import ray 
import os 

# Can be modified
os.environ["RAY_DEDUP_LOGS"] = "0"

print("Ray version :", ray.__version__)
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("MPS Available:", torch.backends.mps.is_available())

torch._dynamo.list_backends()

Ray version : 2.41.0
PyTorch Version: 2.5.1
CUDA Available: False
MPS Available: True


['cudagraphs', 'inductor', 'onnxrt', 'openxla', 'tvm']

## Number of CPUs and GPUs available

In [2]:
import psutil

print("Number of CPUs: ", psutil.cpu_count())

num_cpus = 12
num_gpus = 0

assert num_cpus <= psutil.cpu_count()

Number of CPUs:  12


## Algorithm configuration

In [3]:
from ray.tune.registry import get_trainable_cls
from ray.rllib.policy.policy import PolicySpec

from particle_2d_env import Particle2dEnvironment, MyCallbacks
from config import env_config

ALGO = "PPO"        
FRAMEWORK= "torch"
env = Particle2dEnvironment(env_config)

ppo_config = (
    get_trainable_cls(ALGO).get_default_config()
    .environment(Particle2dEnvironment, env_config=env_config)
    .framework(FRAMEWORK,)
    .api_stack(enable_rl_module_and_learner=True, enable_env_runner_and_connector_v2=True,)
    .callbacks(MyCallbacks)
    .rl_module(
        model_config={
            "fcnet_hiddens" : [128, 128, 128], 
            "use_attention" : True,
        }
    )
    .multi_agent(
        policies= {
            "prey": PolicySpec(
                policy_class=None,  # infer automatically from Algorithm
                observation_space=env.observation_space[0],  # if None infer automatically from env
                action_space=env.action_space[0],  # if None infer automatically from env
                config={"gamma": 0.85},  # use main config plus <- this override here
            ),
            "predator": PolicySpec(
                policy_class=None,
                observation_space=env.observation_space[0],
                action_space=env.action_space[0],
                config={"gamma": 0.85},
            ),
        },
        policy_mapping_fn = lambda id, *arg, **karg: "prey" if env.particule_agents[id].entity_type == 0 else "predator",
        policies_to_train=["prey", "predator"],
        count_steps_by="agent_steps",
    )
    .training(
        num_epochs=10,
        train_batch_size_per_learner=512, 
    )
    .learners(
        num_learners=1,          # or >2
        num_cpus_per_learner=3,  # <- default 1
        num_gpus_per_learner=0,  # <- default 0
    )
    .resources(
        num_cpus_for_main_process=1  # <- default  1
    )
    .env_runners(
        num_env_runners=1, 
        num_envs_per_env_runner=1,
        num_cpus_per_env_runner=2,
        rollout_fragment_length="auto",
        batch_mode= 'complete_episodes', # truncate_episodes or complete_episodes
    )
    .checkpointing(export_native_model_files=True)
)


/opt/homebrew/Caskroom/miniforge/base/envs/collective_env/lib/python3.11/site-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/collective_env/lib/python3.11/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/collective_env/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:134: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/collective_env/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not withi

## Wandb Callback

In [4]:
from ray.air.integrations.wandb import WandbLoggerCallback

# Read the API key from the file to use Wanddb
with open('../project/wandb_api_key.txt', 'r') as file:
    api_key = file.read().strip()
tune_callbacks = [
    WandbLoggerCallback(                   
        project="marl-rllib", 
        group=None,
        api_key=api_key,
        log_config=True,
        upload_checkpoints=False
    ), 
]

## Training

In [5]:
from ray import train, tune
import os

ray.init(
    num_cpus=num_cpus, 
    num_gpus=num_gpus,
    ignore_reinit_error = True,
)

############################################
# Where to save 
############################################
# absolute path + ray_results directory
storage_path=os.getcwd() + "/ray_results"
checkpoint_folder = None # is something like "PPO_2024-12-19_01-09-51"


############################################
# Config
############################################
config_dict = ppo_config.to_dict()

# Environment parameters
#config_dict["env_config"]["friction_regime"] = tune.grid_search(["linear", "quadratic"])
#config_dict["env_config"]["periodical_boundary"] = tune.grid_search([True, False])
config_dict["env_config"]["prey_consumed"] = tune.grid_search([True, False])

#config_dict["env_config"]["num_food_patch"] = tune.grid_search([0, 2])

# RLlib parameters
#config_dict["train_batch_size_per_learner"] = tune.grid_search([256, 512, 1024])


############################################
# Build the Tuner
############################################
if checkpoint_folder is None : 
    tuner = tune.Tuner(
        trainable = ALGO,                                     # Defined before
        param_space=config_dict,                              # Defined before
        run_config=train.RunConfig(    
            storage_path=storage_path,
            stop={"training_iteration": 1500},
            verbose=3,
            callbacks=tune_callbacks,
            checkpoint_config=train.CheckpointConfig(         
                checkpoint_at_end=True,
                checkpoint_frequency=100,
            ),
        ),
    )
else:  # If we start a training that failed
    path = storage_path + "/" + checkpoint_folder
    # Restore the training
    tuner = tune.Tuner.restore(
        trainable = ALGO,
        path = path, 
        resume_unfinished=True, 
        resume_errored=True,
        restart_errored=False,
    )
    

# Run the experiment 
results = tuner.fit()

ray.shutdown()


2025-02-01 13:31:38,723	INFO worker.py:1841 -- Started a local Ray instance.
2025-02-01 13:31:39,209	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


2025-02-01 13:31:39,228	WARNING algorithm_config.py:4702 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
2025-02-01 13:31:39,230	WARNING algorithm_config.py:4702 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
2025-02-01 13:31:39,231	WARNING algorithm_config.py:4702 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want t

2025-02-01 13:32:05,061	WARNING tensorboardx.py:265 -- You are trying to log an invalid value (ray/tune/env_runners/episode_videos_best=[<wandb.sdk.data_types.video.Video object at 0x3e4697b10>]) via TBXLoggerCallback!
(PPO(env=<class 'particle_2d_env.Particle2dEnvironment'>; env-runners=1; learners=1; multi-agent=True) pid=9602) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/Users/tanguy/Code/Finebouche/collective_behavior/project/ray_results/PPO_2025-02-01_13-31-39/PPO_Particle2dEnvironment_7c04b_00000_0_prey_consumed=True_2025-02-01_13-31-39/checkpoint_000000)
(MultiAgentEnvRunner pid=9612) 2025-02-01 13:38:10,367	ERROR actor_manager.py:187 -- Worker exception caught during `apply()`: Agent 24 acted and then got truncated, but did NOT receive a last (truncation) observation, required for e.g. value function bootstrapping!
(MultiAgentEnvRunner pid=9612) Traceback (most recent call last):
(MultiAgentEnvRunner pid=9612)   File "/opt/homebrew/Caskroom/miniforge/b

(raylet) A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: ffffffffffffffff24d3dc8ef8d4f6a7cfacb44401000000 Worker ID: 1519c1157d56a9d86884dc5de82064a6f919b895c29cabdb887383d4 Node ID: c2fed0cd03b5a3c90434566716fc84270f2a54145d53f9c7eeeddc76 Worker IP address: 127.0.0.1 Worker port: 50345 Worker PID: 9612 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker exits unexpectedly. Worker exits with an exit code 1.


(PPO(env=<class 'particle_2d_env.Particle2dEnvironment'>; env-runners=1; learners=1; multi-agent=True) pid=9602) 2025-02-01 13:39:11,467	ERROR actor_manager.py:815 -- Ray error (The actor 24d3dc8ef8d4f6a7cfacb44401000000 is unavailable: The actor is temporarily unavailable: IOError: Fail all inflight tasks due to actor state change.. The task may or maynot have been executed on the actor.), taking actor 1 out of service.
(PPO(env=<class 'particle_2d_env.Particle2dEnvironment'>; env-runners=1; learners=1; multi-agent=True) pid=9602) 2025-02-01 13:39:11,467	ERROR actor_manager.py:646 -- The actor 24d3dc8ef8d4f6a7cfacb44401000000 is unavailable: The actor is temporarily unavailable: IOError: Fail all inflight tasks due to actor state change.. The task may or maynot have been executed on the actor.
(PPO(env=<class 'particle_2d_env.Particle2dEnvironment'>; env-runners=1; learners=1; multi-agent=True) pid=9602) NoneType: None
(PPO(env=<class 'particle_2d_env.Particle2dEnvironment'>; env-runn

...

(raylet) A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: ffffffffffffffff24d3dc8ef8d4f6a7cfacb44401000000 Worker ID: db89c6211c37cac60791bc61ee4a96efc12ce2039d263f375144d062 Node ID: c2fed0cd03b5a3c90434566716fc84270f2a54145d53f9c7eeeddc76 Worker IP address: 127.0.0.1 Worker port: 52752 Worker PID: 10070 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker exits unexpectedly. Worker exits with an exit code 1.


(PPO(env=<class 'particle_2d_env.Particle2dEnvironment'>; env-runners=1; learners=1; multi-agent=True) pid=9602) 2025-02-01 14:41:09,657	ERROR actor_manager.py:815 -- Ray error (The actor 24d3dc8ef8d4f6a7cfacb44401000000 is unavailable: The actor is temporarily unavailable: IOError: Fail all inflight tasks due to actor state change.. The task may or maynot have been executed on the actor.), taking actor 1 out of service.
(PPO(env=<class 'particle_2d_env.Particle2dEnvironment'>; env-runners=1; learners=1; multi-agent=True) pid=9602) 2025-02-01 14:41:09,657	ERROR actor_manager.py:646 -- The actor 24d3dc8ef8d4f6a7cfacb44401000000 is unavailable: The actor is temporarily unavailable: IOError: Fail all inflight tasks due to actor state change.. The task may or maynot have been executed on the actor.
(PPO(env=<class 'particle_2d_env.Particle2dEnvironment'>; env-runners=1; learners=1; multi-agent=True) pid=9602) NoneType: None
(PPO(env=<class 'particle_2d_env.Particle2dEnvironment'>; env-runn

(raylet) A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: ffffffffffffffff24d3dc8ef8d4f6a7cfacb44401000000 Worker ID: 29fa4c24429b55684b80186f813947a96fe5e51c45569e6f6248d9cc Node ID: c2fed0cd03b5a3c90434566716fc84270f2a54145d53f9c7eeeddc76 Worker IP address: 127.0.0.1 Worker port: 53510 Worker PID: 12389 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker exits unexpectedly. Worker exits with an exit code 1.


(PPO(env=<class 'particle_2d_env.Particle2dEnvironment'>; env-runners=1; learners=1; multi-agent=True) pid=9602) 2025-02-01 14:42:20,772	ERROR actor_manager.py:815 -- Ray error (The actor 24d3dc8ef8d4f6a7cfacb44401000000 is unavailable: The actor is temporarily unavailable: RpcError: RPC Error message: Cancelling all calls; RPC Error details: . The task may or maynot have been executed on the actor.), taking actor 1 out of service.
(PPO(env=<class 'particle_2d_env.Particle2dEnvironment'>; env-runners=1; learners=1; multi-agent=True) pid=9602) 2025-02-01 14:42:20,772	ERROR actor_manager.py:646 -- The actor 24d3dc8ef8d4f6a7cfacb44401000000 is unavailable: The actor is temporarily unavailable: RpcError: RPC Error message: Cancelling all calls; RPC Error details: . The task may or maynot have been executed on the actor.
(PPO(env=<class 'particle_2d_env.Particle2dEnvironment'>; env-runners=1; learners=1; multi-agent=True) pid=9602) NoneType: None
(PPO(env=<class 'particle_2d_env.Particle2dE

KeyboardInterrupt: 

In [ ]:
ray.nodes()[0]

ray.shutdown()